In [4]:
import os
import glob
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy.stats import zscore
import torch
import torch.nn as nn 
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, cohen_kappa_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import torcheeg


In [5]:
all_files = glob.glob(r"C:\\Users\\HP\\Desktop\\LSTM\\data\\*.csv")

print(f"เจอไฟล์ทั้งหมด {len(all_files)} ไฟล์:")
for f in all_files:
    print("  →", f.split("\\")[-1])  # แสดงแค่ชื่อไฟล์

# ส่วนที่เหลือเหมือนเดิมทุกอย่าง
dfs = []
for f in all_files:
    df = pd.read_csv(f, header=None)
    df.columns = ['Time', '1', '2', '3', '4', '5', '6', '7', '8', 'Annotation']
    dfs.append(df)
data = pd.concat(dfs, ignore_index=True)
print(f"รวมข้อมูลทั้งหมด: {len(data):,} แถว จาก {len(all_files)} คืน")

เจอไฟล์ทั้งหมด 5 ไฟล์:
  → SN001.csv
  → SN002.csv
  → SN003.csv
  → SN005.csv
  → SN006.csv


C:\Users\HP\AppData\Local\Temp\ipykernel_33088\1782314397.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, header=None)
C:\Users\HP\AppData\Local\Temp\ipykernel_33088\1782314397.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, header=None)
C:\Users\HP\AppData\Local\Temp\ipykernel_33088\1782314397.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, header=None)
C:\Users\HP\AppData\Local\Temp\ipykernel_33088\1782314397.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, header=None)
C:\Users\HP\AppData\Local\Temp\ipykernel_33088\1782314397.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, header=None)


รวมข้อมูลทั้งหมด: 34,500,869 แถว จาก 5 คืน


In [ ]:
# ----------------------------- ตั้งค่าพารามิเตอร์ -----------------------------
OUTPUT_DIR = r"C:\\Users\\HP\\Desktop\\LSTM\\data\\eeg_custom_10s"  # ชื่อโฟลเดอร์ใหม่เพื่อไม่ทับ
SELECT_CHANNEL = '2'  # เปลี่ยนได้ เช่น '1' = EEG F4-M1, '2' = C4-M1, '6' = EOG E1-M2 ฯลฯ

SAMPLING_RATE = 256          # Hz - ข้อมูลของคุณ
EPOCH_SEC = 10               # วินาที
EPOCH_SIZE = EPOCH_SEC * SAMPLING_RATE  # = 2660 samples ต่อ epoch

# Channel map (สำหรับ reference เท่านั้น ไม่ได้ใช้ใน code แต่ดีไว้ดู)
channel_map = {
    "1": "EEG F4-M1",
    "2": "EEG C4-M1",
    "3": "EEG O2-M1",
    "4": "EEG C3-M2",
    "5": "EMG chin",
    "6": "EOG E1-M2",
    "7": "EOG E2-M2",
    "8": "ECG"
}

# Label mapping ตามที่คุณให้มา (สำคัญมาก – ต้องตรงกับตอน train)
label_map = {
    'Sleep stage N1': 0,
    'Sleep stage N2': 1,
    'Sleep stage N3': 2,
    'Sleep stage R': 3,
    'Sleep stage W': 4,
    # เพิ่มกรณีที่อาจเจอในข้อมูลจริง
    'N1': 0, 'N2': 1, 'N3': 2, 'REM': 3, 'Wake': 4,
    'W': 4, 'R': 3,
    'Sleep stage ?': -1,
    'Movement time': -1,
    '?': -1
}
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# วนลูปทีละไฟล์อีกครั้งเพื่อสร้าง .npz (จำเป็นเพราะต้องรู้ subject)
for csv_file in tqdm(all_files, desc="Creating .npz files"):
    subject_name = os.path.basename(csv_file).rsplit('.', 1)[0]
    
    # โหลดไฟล์เดิมอีกครั้ง (เพราะ data ใหญ่รวมแล้วแยก subject ไม่ได้แล้ว)
    df = pd.read_csv(csv_file, header=None)
    df.columns = ['Time', '1', '2', '3', '4', '5', '6', '7', '8', 'Annotation']
    
    signal = df[SELECT_CHANNEL].values.astype(np.float32)
    annotations = df['Annotation'].values
    
    total_samples = len(signal)
    if total_samples < EPOCH_SIZE:
        print(f"  ข้าม {subject_name}: ข้อมูลสั้นเกินไป")
        continue
    
    # ตัดให้ครบ epoch
    new_length = (total_samples // EPOCH_SIZE) * EPOCH_SIZE
    signal = signal[:new_length]
    annotations = annotations[:new_length]
    
    X = []
    y = []
    
    for i in range(len(signal) // EPOCH_SIZE):
        start = i * EPOCH_SIZE
        ann_raw = str(annotations[start]).strip()
        
        if ann_raw not in label_map:
            continue
        label = label_map[ann_raw]
        if label == -1:
            continue
        
        epoch = zscore(signal[start:start + EPOCH_SIZE])
        X.append(epoch)
        y.append(label)
    
    if len(X) == 0:
        print(f"  ข้าม {subject_name}: ไม่มี epoch ที่มี label ถูกต้อง")
        continue
    
    X = np.array(X)[:, :, np.newaxis]  # (n_epochs, 2660, 1)
    y = np.array(y)
    
    npz_path = os.path.join(OUTPUT_DIR, f"{subject_name}.npz")
    np.savez(npz_path, X=X, y=y)
    
    counts = np.unique(y, return_counts=True)
    counts_str = ", ".join([f"{int(k)}:{v}" for k, v in zip(counts[0], counts[1])])
    print(f"  ✓ {subject_name}.npz → {len(X)} epochs | Labels: {counts_str}")  

print(f"\nเสร็จสิ้น! สร้างไฟล์ .npz ทั้งหมดเรียบร้อย")
print(f"โฟลเดอร์: {OUTPUT_DIR}")
print("พร้อมนำไป train โมเดล เช่น DeepSleepNet, TinySleepNet, หรือ LSTM ของคุณเองได้เลย!")

Creating .npz files:   0%|          | 0/5 [00:00<?, ?it/s]C:\Users\HP\AppData\Local\Temp\ipykernel_33088\764091959.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file, header=None)
Creating .npz files:  20%|██        | 1/5 [00:07<00:28,  7.20s/it]

  ✓ SN001.npz → 2562 epochs | Labels: 0:327, 1:1290, 2:69, 3:423, 4:453


C:\Users\HP\AppData\Local\Temp\ipykernel_33088\764091959.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file, header=None)
Creating .npz files:  40%|████      | 2/5 [00:13<00:20,  6.73s/it]

  ✓ SN002.npz → 2568 epochs | Labels: 0:201, 1:927, 2:672, 3:522, 4:246


C:\Users\HP\AppData\Local\Temp\ipykernel_33088\764091959.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file, header=None)
Creating .npz files:  60%|██████    | 3/5 [00:20<00:13,  6.91s/it]

  ✓ SN003.npz → 2862 epochs | Labels: 0:243, 1:504, 2:450, 3:456, 4:1209


C:\Users\HP\AppData\Local\Temp\ipykernel_33088\764091959.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file, header=None)
Creating .npz files:  80%|████████  | 4/5 [00:27<00:06,  6.94s/it]

  ✓ SN005.npz → 2877 epochs | Labels: 0:423, 1:1023, 2:447, 3:381, 4:603


C:\Users\HP\AppData\Local\Temp\ipykernel_33088\764091959.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file, header=None)
Creating .npz files: 100%|██████████| 5/5 [00:34<00:00,  6.85s/it]

  ✓ SN006.npz → 2598 epochs | Labels: 0:459, 1:1050, 2:498, 3:195, 4:396

เสร็จสิ้น! สร้างไฟล์ .npz ทั้งหมดเรียบร้อย
โฟลเดอร์: C:\\Users\\HP\\Desktop\\LSTM\\data\\eeg_custom_10s
พร้อมนำไป train โมเดล เช่น DeepSleepNet, TinySleepNet, หรือ LSTM ของคุณเองได้เลย!
